<a href="https://colab.research.google.com/github/EverlynAsiko/Neural_Machine_Translation_for_African_Languages/blob/main/LugandaLuhya_Multilingual_NMT_results1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual neural machine translation.

For this case, we shall to a many-to-one translation:
{Luganda, Luhya} to English. 

In [ ]:
# Linking to drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Importing needed libraries for preprocessing and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title Default title text
# Install Pytorch with GPU support v1.8.0.
! pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.


In [ ]:
# Filtering warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the drive
import os
os.chdir("/content/gdrive/Shared drives/NMT_for_African_Language")

In [ ]:
# Setting source and target languages
source_language = "en"
target_language = "lg_lh"

os.environ["src"] = source_language 
os.environ["tgt"] = target_language

## Data preprocessing

In [ ]:
! head Luganda/train.*
! head Luganda/dev.*

==> Luganda/train.bpe.en <==
Ev@@ en@@ tually , however , the tru@@ ths I learned from the Bible began to sin@@ k deep@@ er into my heart . I real@@ ized that if I wanted to serve Jehovah , I had to change my pol@@ it@@ ical view@@ poin@@ ts and associ@@ ations .
At last , I have the st@@ able family life that I always cr@@ av@@ ed , and I have the loving Father that I always wanted .
I was a new husband , only 25 years old and very in@@ experienced , but off we went with confidence in Jehovah .
What can you do to show these de@@ a@@ f brothers personal attention ?
R@@ ef@@ er@@ r@@ ing to what the rul@@ er@@ ship of God’s Son will accompl@@ ish , Isaiah 9 : 7 says : “ The very z@@ eal of Jehovah of arm@@ ies will do this . ”
Jesus is the m@@ igh@@ ti@@ est of all of Jehovah’s spirit sons .
The ste@@ ad@@ f@@ ast example set by J@@ ac@@ o@@ b and R@@ ac@@ he@@ l no doubt had a powerful effect on their son Joseph , influ@@ enc@@ ing how he would hand@@ le t@@ ests of his own faith .
Whe

In [ ]:
! head Luhyia/train.*
! head Luhyia/dev.*

==> Luhyia/train.bpe.en <==
Then Pilate entered the P@@ ra@@ et@@ or@@ i@@ um again , called Jesus , and said to Him , “ Are You the King of the Jews ? ”
If anyone th@@ in@@ ks himself to be a prophet or spirit@@ ual , let him ac@@ knowledge that the things which I write to you are the commandments of the Lord .
E@@ very br@@ an@@ ch in Me that does not bear fruit He tak@@ es away ; and every br@@ an@@ ch that be@@ ars fruit He pr@@ un@@ es , that it may bear more fruit .
D@@ em@@ et@@ ri@@ us has a good testimony from all , and from the truth its@@ el@@ f . And we also bear witness , and you know that our testimony is true .
And supp@@ er being ended , the devil having already put it into the heart of Judas Is@@ c@@ ari@@ ot , Simon ’ s son , to betr@@ ay Him ,
im@@ pl@@ or@@ ing us with much ur@@ gen@@ c@@ y that we would receive the gift and the fel@@ low@@ ship of the minis@@ ter@@ ing to the saints .
It is written in the prophets , ‘ And they shall all be taught by G@@ od@@ . ’ Th

In [ ]:
pre = '/content/gdrive/Shared drives/NMT_for_African_Language/'
# Train data source
filenames = [pre+'Luganda/train.en',pre+'Luhyia/train.en']

# Train data target
filenames2 = [pre+'Luganda/train.lg',pre+'Luhyia/train.lh']

# Dev data source
file1 = [pre+'Luganda/dev.en',pre+'Luhyia/dev.en']

# Dev data target
file2 = [pre+'Luganda/dev.lg',pre+'Luhyia/dev.lh']

In [ ]:
pre2 = '/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual/'
trun_file1 = [pre2+'train_trunc.en1',pre+'Luhyia/train.en']
trun_file2 = [pre2+'train.lg',pre+'Luhyia/train.lh']

In [ ]:
# Changing to Multilingual2 directory
os.chdir("/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2")

In [ ]:
# Procedure to create concatenated files
def create_file(x,filename):
  # Open filename in write mode
  with open(filename, 'w') as outfile:
      for names in x:
          # Open each file in read mode
          with open(names) as infile:
              # read the data and write it in file3
              outfile.write(infile.read())
          outfile.write("\n")

In [ ]:
# Creating multilingual files
create_file(filenames,'train.en')
create_file(filenames2,'train.lg_lh')
create_file(file1,'dev.en')
create_file(file2,'dev.lg_lh')

### BPE codes

In [ ]:
#! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Processing /content/gdrive/Shareddrives/NMT_for_African_Language/Multilingual2/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 15.3 MB 92 kB/s 
     |████████████████████████████████| 7.1 MB 21.5 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 375 kB 62.5 MB/s 
     |████████████████████████████████| 231 kB 68.3 MB/s 
     |████████████████████████████████| 105 kB 72.7 MB/s 
     |████████████████████████████████| 743 kB 58.7 MB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 


In [ ]:
# Apply BPE splits to the development and test data.
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py train.bpe.$src train.bpe.$tgt --output_path vocab.txt

In [ ]:
# Applying BPE to tests
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test1.$src > test.bpe.en1
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test1.lh > test.bpe.lh

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test2.$src > test.bpe.en2
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test2.lg > test.bpe.lg

In [ ]:
# Some output
! echo "BPE Sentences"
! tail -n 5 test.bpe.lh
! echo "Combined BPE Vocab"
! tail -n 10 vocab.txt

BPE Sentences
N@@ asi , n@@ ir@@ e@@ eba end@@ i , ‘ N@@ i@@ we w@@ ina , Omwami ? ’ Omwoyo okwo , n@@ i@@ kum@@ bo@@ ol@@ ela ku@@ r@@ i , ‘ N@@ is@@ ie Yesu owa N@@ az@@ ar@@ et@@ i ow@@ os@@ aan@@ d@@ inj@@ ia . ’
sh@@ ich@@ ila , omuk@@ h@@ aan@@ a@@ we omut@@ el@@ wa , ow@@ em@@ iy@@ ika ek@@ hum@@ i n@@ ach@@ ib@@ ili yali n@@ any@@ iranga . Ne olwa yali nat@@ s@@ it@@ s@@ anga , abandu , bam@@ w@@ ib@@ um@@ bak@@ h@@ wo okhur@@ ula mut@@ sim@@ b@@ eka t@@ si@@ osi .
Ne olwa k@@ ab@@ is@@ ibwa mbu kh@@ uk@@ ho@@ y@@ ile okh@@ ut@@ si@@ ila , mum@@ e@@ el@@ i okh@@ uula I@@ t@@ ali@@ a , ba@@ h@@ aana Pa@@ ulo nende abab@@ o@@ he , b@@ andi kh@@ um@@ us@@ inj@@ il@@ ili w@@ el@@ i@@ he J@@ ul@@ i@@ asi o@@ we@@ ing@@ '@@ anda eya , esh@@ ir@@ oma ey@@ il@@ ang@@ wa mbu , “ I@@ ng@@ '@@ anda ey@@ il@@ ind@@ anga , Om@@ ur@@ uc@@ h@@ i . ”
Ol@@ uny@@ um@@ akh@@ wo , abaku@@ uka be@@ f@@ we , ab@@ abu@@ kula li@@ he@@ ema el@@ o okhur@@ ula kh@@ ub@@ as@@ abwe , bal@@ ich@@ inga , ok

In [ ]:
! tail train.*
! tail dev.*

==> train.bpe.en <==
But if anyone lo@@ ves God , this one is known by Him .
And the second is like it : ‘ You shall love your neigh@@ b@@ or as yourself . ’
until the day in which He was taken up , after He through the H@@ ol@@ y Sp@@ irit had given command@@ ments to the apostles whom He had cho@@ s@@ en ,
For what if some did not believe ? W@@ ill their un@@ belie@@ f make the faith@@ ful@@ ness of God without effect ?
And when you go into a hou@@ se@@ hold , gre@@ et it .
And a very great mul@@ t@@ itude sp@@ read their clo@@ th@@ es on the ro@@ ad ; others c@@ ut down br@@ an@@ ch@@ es from the tre@@ es and sp@@ read them on the ro@@ ad .
And we heard this vo@@ ice which came from heaven when we were with Him on the holy mo@@ un@@ tain .
O@@ r those e@@ igh@@ t@@ een on whom the tower in S@@ il@@ o@@ am f@@ ell and kill@@ ed them , do you think that they were wor@@ se sinn@@ ers than all other men who dw@@ el@@ t in Jerusalem ?
For I be@@ ar him witness that he has a great z@@ eal

## Modelling

### Baseline

In [ ]:
#@title
name = '%s%s' % (target_language, source_language)

# Create the config
config = """
name: "{target_language}{source_language}_reverse_transformer"

data:
    src: "{target_language}"
    trg: "{source_language}"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 1000
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                  # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 2500         # TODO: Set to at least once per epoch.
    logging_freq: 200
    eval_metric: "bleu"
    model_dir: "models/{name}_reverse_transformer"
    overwrite: True 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path="/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2", source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_reverse_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
# Train the model
!cd joeynmt; python3 -m joeynmt train configs/transformer_reverse_$tgt$src.yaml

2021-07-13 10:38:45,835 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-13 10:38:45,865 - INFO - joeynmt.data - Loading training data...
2021-07-13 10:38:50,769 - INFO - joeynmt.data - Building vocabulary...
2021-07-13 10:38:51,077 - INFO - joeynmt.data - Loading dev data...
2021-07-13 10:38:51,123 - INFO - joeynmt.data - Loading test data...
2021-07-13 10:38:51,144 - INFO - joeynmt.data - Data loaded.
2021-07-13 10:38:51,145 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-13 10:38:51,394 - INFO - joeynmt.model - Enc-dec model built.
2021-07-13 10:38:51.575863: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 10:38:53,789 - INFO - joeynmt.training - Total params: 12152320
2021-07-13 10:38:55,963 - INFO - joeynmt.helpers - cfg.name                           : lg_lhen_reverse_transformer
2021-07-13 10:38:55,964 - INFO - joeynmt.helpers - cfg.data.src                       

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/lg_lhen_reverse_transformer/validations.txt"

Steps: 2500	Loss: 228267.98438	PPL: 41.16292	bleu: 2.56686	LR: 0.00030000	*
Steps: 5000	Loss: 199037.65625	PPL: 25.57197	bleu: 5.08604	LR: 0.00030000	*
Steps: 7500	Loss: 185500.95312	PPL: 20.51262	bleu: 6.36613	LR: 0.00030000	*
Steps: 10000	Loss: 176041.46875	PPL: 17.58392	bleu: 8.14781	LR: 0.00030000	*
Steps: 12500	Loss: 170617.84375	PPL: 16.09738	bleu: 8.55822	LR: 0.00030000	*
Steps: 15000	Loss: 165838.31250	PPL: 14.89190	bleu: 9.21606	LR: 0.00030000	*
Steps: 17500	Loss: 160958.62500	PPL: 13.75425	bleu: 10.78737	LR: 0.00030000	*
Steps: 20000	Loss: 157274.68750	PPL: 12.95332	bleu: 11.50072	LR: 0.00030000	*
Steps: 22500	Loss: 154986.70312	PPL: 12.47953	bleu: 12.13082	LR: 0.00030000	*
Steps: 25000	Loss: 155826.45312	PPL: 12.65138	bleu: 11.13397	LR: 0.00030000	
Steps: 27500	Loss: 150788.40625	PPL: 11.65479	bleu: 12.49335	LR: 0.00030000	*
Steps: 30000	Loss: 148554.75000	PPL: 11.23844	bleu: 12.29347	LR: 0.00030000	*
Steps: 32500	Loss: 146919.82812	PPL: 10.94315	bleu: 12.92199	LR: 0.0003000

In [ ]:
# Reloading configuration file
ckpt_number = 32500
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/models/lg_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/lg_lhen_reverse_transformer"', f'model_dir: "models/lg_lhen_reverse_transformer_continued"').replace(
        f'epochs: 30', f'epochs: 18')
        
with open("joeynmt/configs/transformer_{name}_reload.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_lg_lhen_reload.yaml"


name: "lg_lhen_reverse_transformer"

data:
    src: "lg_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer/32500.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # T

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_lg_lhen_reload.yaml

2021-07-13 14:24:21,277 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-13 14:24:21,349 - INFO - joeynmt.data - Loading training data...
2021-07-13 14:24:26,459 - INFO - joeynmt.data - Building vocabulary...
2021-07-13 14:24:27,122 - INFO - joeynmt.data - Loading dev data...
2021-07-13 14:24:27,925 - INFO - joeynmt.data - Loading test data...
2021-07-13 14:24:29,027 - INFO - joeynmt.data - Data loaded.
2021-07-13 14:24:29,027 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-13 14:24:29,224 - INFO - joeynmt.model - Enc-dec model built.
2021-07-13 14:24:29.469826: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 14:24:32,282 - INFO - joeynmt.training - Total params: 12152320
2021-07-13 14:24:40,984 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer/32500.ckpt
2021-07-

In [ ]:
# Output our validation accuracy
! cat "joeynmt/models/lg_lhen_reverse_transformer_continued/validations.txt"

Steps: 35000	Loss: 145461.15625	PPL: 10.68626	bleu: 13.82629	LR: 0.00030000	*
Steps: 37500	Loss: 144813.43750	PPL: 10.57412	bleu: 13.95177	LR: 0.00030000	*
Steps: 40000	Loss: 142993.68750	PPL: 10.26534	bleu: 14.04079	LR: 0.00030000	*
Steps: 42500	Loss: 141620.42188	PPL: 10.03831	bleu: 14.43683	LR: 0.00030000	*
Steps: 45000	Loss: 141061.18750	PPL: 9.94730	bleu: 14.58504	LR: 0.00030000	*
Steps: 47500	Loss: 140957.23438	PPL: 9.93047	bleu: 14.24453	LR: 0.00030000	*
Steps: 50000	Loss: 140635.35938	PPL: 9.87855	bleu: 14.14640	LR: 0.00030000	*
Steps: 52500	Loss: 138489.68750	PPL: 9.53932	bleu: 15.12613	LR: 0.00030000	*
Steps: 55000	Loss: 137841.76562	PPL: 9.43919	bleu: 15.60457	LR: 0.00030000	*
Steps: 57500	Loss: 137239.71875	PPL: 9.34709	bleu: 15.36421	LR: 0.00030000	*
Steps: 60000	Loss: 136941.43750	PPL: 9.30180	bleu: 14.61966	LR: 0.00030000	*
Steps: 62500	Loss: 135767.00000	PPL: 9.12557	bleu: 15.43062	LR: 0.00030000	*


In [ ]:
# Reloading configuration file
ckpt_number = 62500
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/models/lg_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/lg_lhen_reverse_transformer"', f'model_dir: "models/lg_lhen_reverse_transformer_continued2"').replace(
        f'epochs: 30', f'epochs: 6')
        
with open("joeynmt/configs/transformer_{name}_reload2.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_lg_lhen_reload2.yaml"


name: "lg_lhen_reverse_transformer"

data:
    src: "lg_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued/62500.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"    

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_lg_lhen_reload2.yaml

2021-07-14 12:56:02,417 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 12:56:02,506 - INFO - joeynmt.data - Loading training data...
2021-07-14 12:56:09,114 - INFO - joeynmt.data - Building vocabulary...
2021-07-14 12:56:09,935 - INFO - joeynmt.data - Loading dev data...
2021-07-14 12:56:10,976 - INFO - joeynmt.data - Loading test data...
2021-07-14 12:56:12,352 - INFO - joeynmt.data - Data loaded.
2021-07-14 12:56:12,352 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 12:56:12,597 - INFO - joeynmt.model - Enc-dec model built.
2021-07-14 12:56:12.855030: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-14 12:56:15,938 - INFO - joeynmt.training - Total params: 12152320
2021-07-14 12:56:24,564 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued/62500.ckp

In [ ]:
!cd joeynmt; python -m joeynmt test 'models/lg_lhen_reverse_transformer_continued2/config.yaml'

2021-07-14 14:35:39,152 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 14:35:39,157 - INFO - joeynmt.data - Building vocabulary...
2021-07-14 14:35:39,457 - INFO - joeynmt.data - Loading dev data...
2021-07-14 14:35:39,491 - INFO - joeynmt.data - Loading test data...
2021-07-14 14:35:39,510 - INFO - joeynmt.data - Data loaded.
2021-07-14 14:35:39,534 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 5000 (with beam_size)
2021-07-14 14:35:42,202 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 14:35:42,451 - INFO - joeynmt.model - Enc-dec model built.
2021-07-14 14:35:42,540 - INFO - joeynmt.prediction - Decoding on dev set (/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/dev.bpe.en)...
2021-07-14 14:38:21,830 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-07-14 14:38:21,831 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, w

In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/lg_lhen_reverse_transformer_continued2/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe.lh" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/translation.bpe.lh_en"

2021-07-14 14:40:45,978 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 14:40:48,854 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 14:40:49,116 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation.bpe.lh_en" | sacrebleu "test1.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 10.1 41.5/15.1/6.3/3.1 (BP = 0.961 ratio = 0.962 hyp_len = 25045 ref_len = 26044)


In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/lg_lhen_reverse_transformer_continued/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe.lg" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/translation.bpe.lg_en"

2021-07-14 14:42:54,924 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-14 14:42:57,838 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-14 14:42:58,080 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
#@title
def empty_counter(x):
  # Opening a file
  infile = open(x,"r")
  empty = []
  
  for i,line in enumerate(infile):
    if not line.strip(): 
      empty.append(i)

  return empty

In [ ]:
#@title
# Reference: https://thispointer.com/python-how-to-delete-specific-lines-in-a-file-in-a-memory-efficient-way/
def delete_multiple_lines(original_file, line_numbers):
    """In a file, delete the lines at line number in given list"""
    is_skipped = False
    counter = 0
    # Create name of dummy / temporary file
    dummy_file = original_file + '.bak'
    # Open original file in read only mode and dummy file in write mode
    with open(original_file, 'r') as read_obj, open(dummy_file, 'w') as write_obj:
        # Line by line copy data from original file to dummy file
        for line in read_obj:
            # If current line number exist in list then skip copying that line
            if counter not in line_numbers:
                write_obj.write(line)
            else:
                is_skipped = True
            counter += 1
    # If any line is skipped then rename dummy file as original file
    if is_skipped:
        os.remove(original_file)
        os.rename(dummy_file, original_file)
    else:
        os.remove(dummy_file)

In [ ]:
delete_multiple_lines("test2.en",empty_counter("test2.lg"))

In [ ]:
!cat "translation.bpe.lg_en" | sacrebleu "test2.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 35.1 65.0/42.8/31.8/24.7 (BP = 0.911 ratio = 0.915 hyp_len = 39454 ref_len = 43116)


In [ ]:
# Reloading configuration file
ckpt_number = 77500
reload_config = config.replace(
    f'#load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/models/lg_lhen_transformer/1.ckpt"', 
    f'load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued2/{ckpt_number}.ckpt"').replace(
        f'model_dir: "models/lg_lhen_reverse_transformer"', f'model_dir: "models/lg_lhen_reverse_transformer_continued3"')
        
with open("joeynmt/configs/transformer_{name}_reload3.yaml".format(name=name),'w') as f:
    f.write(reload_config)

In [ ]:
!cat "joeynmt/configs/transformer_lg_lhen_reload3.yaml"


name: "lg_lhen_reverse_transformer"

data:
    src: "lg_lh"
    trg: "en"
    train: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/train.bpe"
    dev:   "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/dev.bpe"
    test:  "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"
    trg_vocab: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued2/77500.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"   

In [ ]:
# Train continued
!cd joeynmt; python3 -m joeynmt train configs/transformer_lg_lhen_reload3.yaml

2021-07-25 07:29:08,461 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-25 07:29:08,526 - INFO - joeynmt.data - Loading training data...
2021-07-25 07:29:13,759 - INFO - joeynmt.data - Building vocabulary...
2021-07-25 07:29:14,503 - INFO - joeynmt.data - Loading dev data...
2021-07-25 07:29:15,213 - INFO - joeynmt.data - Loading test data...
2021-07-25 07:29:15,979 - INFO - joeynmt.data - Data loaded.
2021-07-25 07:29:15,979 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-25 07:29:16,346 - INFO - joeynmt.model - Enc-dec model built.
2021-07-25 07:29:16.590924: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-25 07:29:18,892 - INFO - joeynmt.training - Total params: 12152320
2021-07-25 07:29:29,521 - INFO - joeynmt.training - Loading model from /content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/joeynmt/models/lg_lhen_reverse_transformer_continued2/77500.ck

In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/lg_lhen_reverse_transformer_continued3/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe.lh" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/translation.bpe.lh_en"

2021-07-25 11:47:45,081 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-25 11:47:49,094 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-25 11:47:49,291 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation.bpe.lh_en" | sacrebleu "test1.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 12.6 44.2/17.7/8.3/4.4 (BP = 0.969 ratio = 0.969 hyp_len = 25240 ref_len = 26044)


In [ ]:
!cd joeynmt; python -m joeynmt translate 'models/lg_lhen_reverse_transformer_continued3/config.yaml' < "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/test.bpe.lg" > "/content/gdrive/Shared drives/NMT_for_African_Language/Multilingual2/translation2.bpe.lg_en"

2021-07-25 11:52:49,017 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-07-25 11:52:53,224 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-07-25 11:52:53,426 - INFO - joeynmt.model - Enc-dec model built.


In [ ]:
!cat "translation2.bpe.lg_en" | sacrebleu "test2.en"

sacreBLEU: That's 100 lines that end in a tokenized period ('.')
sacreBLEU: It looks like you forgot to detokenize your test data, which may hurt your score.
sacreBLEU: If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.
BLEU+case.mixed+numrefs.1+smooth.exp+tok.13a+version.1.5.1 = 37.3 66.5/45.0/34.0/26.8 (BP = 0.919 ratio = 0.922 hyp_len = 39750 ref_len = 43116)
